# How the Human Brain Makes Sense of Natural Scenes
## Linearizing Encoding Model - Notebook

#### Import Libs

In [1]:
import os
import numpy as np
import math
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from scipy.stats import pearsonr as corr

In [ ]:
data_dir = 'nsd_data'
parent_submission_dir = 'submission_data'

In [ ]:
device = 'cpu'
device = torch.device(device)

#### Select Subject (1-8):

In [ ]:
subj = 1 # 1, 2, 3, 4, 5, 6, 7, 8

#### Define data paths

In [ ]:
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):
    
    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

    # Create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

args = argObj(data_dir, parent_submission_dir, subj) 

## Load fMRI training data

Load the training split of the fMRI data (from data_dir) for the selected subject (subj), for both hemispheres (LH/RH).

The files each contain a 2-dimensional array of the training stimulus images and corresponding fMRI veriticies.

In [ ]:
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)\n')

print('RH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

## fMRI ROIs

Reigons-of-interest (ROIs) divide the visual cortex into different local groups of vertices, performing similar functions. The files in the 'roi_masks' directory map brain ROIs to vertcies and vertices to fsaverage space, a standardized brain surface map for all subjects.

### Visualize all vertices on a brain surface map

Map all vertices, using the left and right ROI masks provided in the files '\*h.all-vertices_fsaverage_space.npy', to the indicies on the brain surface map in fsaverage space. We then output the graphical brain surface map highlighting the areas that represent the verticies mapped for all ROIs.

#### Select Hemisphere:

In [ ]:
hemisphere = 'left' # 'left', right'

#### Brain Surface Map

In [ ]:
# Load the brain surface map of all vertices
roi_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.all-vertices_fsaverage_space.npy')
fsaverage_all_vertices = np.load(roi_dir)

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_all_vertices,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cool',
    colorbar=False,
    title='All vertices, '+hemisphere+' hemisphere'
    )
view

### Visualize a chosen ROI on a brain surface map

Using the files '\*h.\*_fsaverage_space.npy' we map the vertices of several ROIs from a given class to the brain surface map in fsaverage space, then use the files mapping_\*.npy to select a specific ROI within the class to visualize on the brain surface map. 

#### Select Parameters

In [ ]:
hemisphere = 'left' # 'left', right'
roi = "EBA" #"V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "EBA", "FBA-1", "FBA-2", "mTL-bodies", "OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces", "OPA", "PPA", "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words", "early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"

#### Brain Surface Map

In [ ]:
# Define the ROI class based on the selected ROI
if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
    roi_class = 'prf-visualrois'
elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
    roi_class = 'floc-bodies'
elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
    roi_class = 'floc-faces'
elif roi in ["OPA", "PPA", "RSC"]:
    roi_class = 'floc-places'
elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
    roi_class = 'floc-words'
elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
    roi_class = 'streams'

# Load the ROI brain surface maps
roi_class_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.'+roi_class+'_fsaverage_space.npy')
roi_map_dir = os.path.join(args.data_dir, 'roi_masks',
    'mapping_'+roi_class+'.npy')
fsaverage_roi_class = np.load(roi_class_dir)
roi_map = np.load(roi_map_dir, allow_pickle=True).item()

# Select the vertices corresponding to the ROI of interest
roi_mapping = list(roi_map.keys())[list(roi_map.values()).index(roi)]
fsaverage_roi = np.asarray(fsaverage_roi_class == roi_mapping, dtype=int)

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_roi,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cool',
    colorbar=False,
    title=roi+', '+hemisphere+' hemisphere'
    )
view

## Stimulus images

Images of natural scenes come from the COCO dataset. https://cocodataset.org/#home

The images were pre-split into training and test partitions and correspond to the same training and test splits used for the fMRI data. The number of images within the training and test splits vary between subjects.

### Load stimulus imgaes

In [ ]:
train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(args.data_dir, 'test_split', 'test_images')

# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
print('Training images: ' + str(len(train_img_list)))
print('Test images: ' + str(len(test_img_list)))

## Visualize the fMRI responses to selected training images

### Map all vertices

#### Select Parameters:

In [ ]:
img = 42 #'0-[9841, 9841, 9082, 8779, 9841, 9082, 9841, 8779]' for each subject.
hemisphere = 'left' # 'left', right'

#### Brain Surface Map

In [ ]:
# Load the image
img_dir = os.path.join(train_img_dir, train_img_list[img])
train_img = Image.open(img_dir).convert('RGB')

# Plot the image
plt.figure()
plt.axis('off')
plt.imshow(train_img)
plt.title('Training image: ' + str(img+1));


# Load the brain surface map of all vertices
roi_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.all-vertices_fsaverage_space.npy')
fsaverage_all_vertices = np.load(roi_dir)

# Map the fMRI data onto the brain surface map
fsaverage_response = np.zeros(len(fsaverage_all_vertices))
if hemisphere == 'left':
    fsaverage_response[np.where(fsaverage_all_vertices)[0]] = lh_fmri[img]
elif hemisphere == 'right':
    fsaverage_response[np.where(fsaverage_all_vertices)[0]] = rh_fmri[img]

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_response,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title='All vertices, '+hemisphere+' hemisphere'
    )
view

### Map for chosen individual ROI

#### Select ROI:

In [ ]:
roi = "lateral" #"V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "EBA", "FBA-1", "FBA-2", "mTL-bodies", "OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces", "OPA", "PPA", "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words", "early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"

#### Brain Surface Map

In [ ]:
# Plot the image
plt.figure()
plt.axis('off')
plt.imshow(train_img)
plt.title('Training image: ' + str(img+1));

# Define the ROI class based on the selected ROI
if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
    roi_class = 'prf-visualrois'
elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
    roi_class = 'floc-bodies'
elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
    roi_class = 'floc-faces'
elif roi in ["OPA", "PPA", "RSC"]:
    roi_class = 'floc-places'
elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
    roi_class = 'floc-words'
elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
    roi_class = 'streams'

# Load the ROI brain surface maps
challenge_roi_class_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.'+roi_class+'_challenge_space.npy')
fsaverage_roi_class_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.'+roi_class+'_fsaverage_space.npy')
roi_map_dir = os.path.join(args.data_dir, 'roi_masks',
    'mapping_'+roi_class+'.npy')
challenge_roi_class = np.load(challenge_roi_class_dir)
fsaverage_roi_class = np.load(fsaverage_roi_class_dir)
roi_map = np.load(roi_map_dir, allow_pickle=True).item()

# Select the vertices corresponding to the ROI of interest
roi_mapping = list(roi_map.keys())[list(roi_map.values()).index(roi)]
challenge_roi = np.asarray(challenge_roi_class == roi_mapping, dtype=int)
fsaverage_roi = np.asarray(fsaverage_roi_class == roi_mapping, dtype=int)

# Map the fMRI data onto the brain surface map
fsaverage_response = np.zeros(len(fsaverage_roi))
if hemisphere == 'left':
    fsaverage_response[np.where(fsaverage_roi)[0]] = \
        lh_fmri[img,np.where(challenge_roi)[0]]
elif hemisphere == 'right':
    fsaverage_response[np.where(fsaverage_roi)[0]] = \
        rh_fmri[img,np.where(challenge_roi)[0]]

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_response,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title=roi+', '+hemisphere+' hemisphere'
    )
view

## Building the Linearizing Encoding models

Taking the training data that we have loaded and visualized, we can now begin pre-processing it before training and fitting our regression models on it.

We first need to split the available data into training and validation partitions to allow for model performance evaluation after fitting our models. Normally we would use one of sklearn's cross-validation method implementations that provide built in functionality for splitting the data before training and validating the model. However, our training data consists of both stimulus images and their corresponding fMRI responses, also requiring unique pre-processing, (extracting and downsampling image features). Therefore we define a series of functions to; pre-process the images; create a data structure to store them;   randomly select a given proportion (10%) of image indicies for validation, and use the selected indexes to retrieve the relevant fMRI data, before repeating with a different random seed and therefore image indicies for each fold. We then average the evaluation (correlation) scores from each fold to give a final cross-validated average score for each hemisphere (LH, RH).

We first define the transform to be applied to the images prior to inputting them to AlexNet. We use a standard preprocessing pipeline, synonymous with typical/common computer vision preprocessing, for formatting the images for optimal feature extraction by the CNN.

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)), # resize the images to 224x24 pixels
    transforms.ToTensor(), # convert the images to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalize the images color channels
])

Next we define a custom implementation of the ImageDataset class from PyTorch, for storing the images from each partition before passing them to AlexNet, where each image is loaded and pretranformed before being returned to be used for feature extraction.

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        # Preprocess the image and send it to the chosen device ('cpu')
        if self.transform:
            img = self.transform(img).to(device)
        return img

We then load the pre-trained AlexNet model, selecting the feature extraction layer which we wish to use for training our model on. 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet')
model.to(device) # send the model to the chosen device ('cpu')
model.eval() # set the model to evaluation mode, since we are not training it

#train_nodes, _ = get_graph_node_names(model)
#print(train_nodes)

#### Select feature layer: 

In [ ]:
model_layer = "features.2" #["features.2", "features.5", "features.7", "features.9", "features.12", "classifier.2", "classifier.5", "classifier.6"] {allow-input: true}
feature_extractor = create_feature_extractor(model, return_nodes=[model_layer])

The feature layer from AlexNet is very large, so we downsample the output using the dimensionality reduction technique, Principal Component Analysis (PCA). The below method is used to fit the PCA on the training images features, and then downsample the training, validation and test images features.

'IncrementalPCA()' is used to partially fit the pca on the images in batches, saving memory as jupyter notebook has RAM limitations. 

In [ ]:
def fit_pca(feature_extractor, dataloader):

    # Define PCA parameters
    pca = IncrementalPCA(n_components=100, batch_size=300)

    # Fit PCA to batch
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Extract features
        ft = feature_extractor(d)
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # Fit PCA to batch
        pca.partial_fit(ft.detach().cpu().numpy())
    return pca

extract_features() then takes the outputs of the above methods and cells as parameters, (feature_extractor for extracting image features from chosen 'model_layer', 'dataloader' containing 'ImageDataset' for extraction, and 'pca' for downsampling extracted features), returning the downsampled imaged features from AlexNet.

In [ ]:
def extract_features(feature_extractor, dataloader, pca):

    features = []
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Extract features
        ft = feature_extractor(d)
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # Apply PCA transform
        ft = pca.transform(ft.cpu().detach().numpy())
        features.append(ft)
    return np.vstack(features)

### Cross-validation

We now perform the cross-validation. This involves setting (incrementing) the random seed and then splitting the training data. The images corresponding to the indexes selected for each partition are then loaded into the relevant ImageDataset using the DataLoader class (also from PyTorch).

In [ ]:
def cross_validation(model_parameters, folds):
    lh_correlation_arrays = []
    rh_correlation_arrays = []

    for i in range (folds):
        rand_seed = i
        np.random.seed(rand_seed)

        # Calculate how many stimulus images correspond to 90% of the training data
        num_train = int(np.round(len(train_img_list) / 100 * 90))
        # Shuffle all training stimulus images
        idxs = np.arange(len(train_img_list))
        np.random.shuffle(idxs)
        # Assign 90% of the shuffled stimulus images to the training partition,
        # and 10% to the validation partition
        idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]

        print('Training stimulus images: ' + format(len(idxs_train)))
        print('\nValidation stimulus images: ' + format(len(idxs_val)))

        batch_size = 300
        # Get the paths of all image files
        train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))
        test_imgs_paths = sorted(list(Path(test_img_dir).iterdir()))

        # The DataLoaders contain the ImageDataset class
        train_imgs_dataloader = DataLoader(
            ImageDataset(train_imgs_paths, idxs_train, transform), 
            batch_size=batch_size
        )
        val_imgs_dataloader = DataLoader(
            ImageDataset(train_imgs_paths, idxs_val, transform), 
            batch_size=batch_size
        )

        lh_fmri_train = lh_fmri[idxs_train]
        lh_fmri_val = lh_fmri[idxs_val]
        rh_fmri_train = rh_fmri[idxs_train]
        rh_fmri_val = rh_fmri[idxs_val]

        pca = fit_pca(feature_extractor, train_imgs_dataloader)

        features_train = extract_features(feature_extractor, train_imgs_dataloader, pca)
        features_val = extract_features(feature_extractor, val_imgs_dataloader, pca)

        print('\nTraining images features:')
        print(features_train.shape)
        print('(Training stimulus images × PCA features)')

        print('\nValidation images features:')
        print(features_val.shape)
        print('(Validation stimulus images × PCA features)')

        # Fit linear regressions on the training data
        if(model_parameters[0] == "default"):
            reg_lh = LinearRegression().fit(features_train, lh_fmri_train)
            reg_rh = LinearRegression().fit(features_train, rh_fmri_train)
        elif(model_parameters[0] == "Lasso"):
            reg_lh = Lasso(alpha = model_parameters[1]).fit(features_train, lh_fmri_train)
            reg_rh = Lasso(alpha = model_parameters[1]).fit(features_train, rh_fmri_train)
        elif(model_parameters[0] == "Ridge"):
            reg_lh = Ridge(alpha = model_parameters[1]).fit(features_train, lh_fmri_train)
            reg_lh = Ridge(alpha = model_parameters[1]).fit(features_train, rh_fmri_train)
        # Use fitted linear regressions to predict the validation fMRI data
        lh_fmri_val_pred = reg_lh.predict(features_val)
        rh_fmri_val_pred = reg_rh.predict(features_val)

        # Empty correlation array of shape: (LH vertices)
        lh_correlation = np.zeros(lh_fmri_val_pred.shape[1])
        # Correlate each predicted LH vertex with the corresponding ground truth vertex
        for v in tqdm(range(lh_fmri_val_pred.shape[1])):
            lh_correlation[v] = corr(lh_fmri_val_pred[:,v], lh_fmri_val[:,v])[0]
        lh_correlation_arrays.append(lh_correlation)

        # Empty correlation array of shape: (RH vertices)
        rh_correlation = np.zeros(rh_fmri_val_pred.shape[1])
        # Correlate each predicted RH vertex with the corresponding ground truth vertex
        for v in tqdm(range(rh_fmri_val_pred.shape[1])):
            rh_correlation[v] = corr(rh_fmri_val_pred[:,v], rh_fmri_val[:,v])[0]
        rh_correlation_arrays.append(rh_correlation)

    lh_average_correlation = []    
    rh_average_correlation = []

    for i in range(len(lh_correlation_arrays[0])):
        lh_num = 0
        rh_num = 0
        for j in range (len(lh_correlation_arrays)):
            lh_num += lh_correlation_arrays[j][i];
            rh_num += rh_correlation_arrays[j][i];
        lh_average_correlation.append(lh_num / len(lh_correlation_arrays));
        rh_average_correlation.append(rh_num / len(rh_correlation_arrays));
        
    return lh_average_correlation, rh_average_correlation

For standard 'cross_validation()' with default parameters, each fold takes ~5 mins to fully execute. This is due to the overhead of extracting features and fitting the PCA on the current train-validation splits every fold. While this improves the accuracy of the image features used for training, it significantly slows down runtime.

Therefore we implement a 'fast_cross_validation()' method used for hyperparameter tuning, where we extract and downsample features for all images prior, (storing the full image 'features' set as a global variable). Each fold then simply randomly selects the 'idxs' (using a new random seed) and uses them to partition the 'features' into 'features_train' and 'features_val', and each hemisphere's, '*h_fmri' vertex responses into '*h_fmri_train' and '*h_fmri_val'.\
We then just repeat the same process of fitting the model (using 'model_parameters') and predicting fMRI responses, evaluating the predictions agasint the unseen ground truth fMRI data from the validation partition. We then return an array of the average correlation scores for each vertex across all the cross validation folds. 

In [ ]:
def fast_cross_validation(model_parameters, folds):
    num_train = int(np.round(len(train_img_list) / 100 * 90))
    
    lh_correlation_arrays = []
    rh_correlation_arrays = []
    
    for i in tqdm(range(folds)):
        rand_seed = i
        np.random.seed(rand_seed)
        
        idxs = np.arange(len(train_img_list))
        np.random.shuffle(idxs)
        idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]

        features_train = features[idxs_train] 
        features_val = features[idxs_val]

        lh_fmri_train = lh_fmri[idxs_train]
        lh_fmri_val = lh_fmri[idxs_val]
        rh_fmri_train = rh_fmri[idxs_train]
        rh_fmri_val = rh_fmri[idxs_val]

        # Fit linear regressions on the training data
        if(model_parameters[0] == "default"):
            reg_lh = LinearRegression().fit(features_train, lh_fmri_train)
            reg_rh = LinearRegression().fit(features_train, rh_fmri_train)
        elif(model_parameters[0] == "Lasso"):
            reg_lh = Lasso(alpha = model_parameters[1]).fit(features_train, lh_fmri_train)
            reg_rh = Lasso(alpha = model_parameters[1]).fit(features_train, rh_fmri_train)
        elif(model_parameters[0] == "Ridge"):
            reg_lh = Ridge(alpha = 1 - model_parameters[1]).fit(features_train, lh_fmri_train)
            reg_rh = Ridge(alpha = 1 - model_parameters[1]).fit(features_train, rh_fmri_train)
            # Use fitted linear regressions to predict the validation fMRI data
        lh_fmri_val_pred = reg_lh.predict(features_val)
        rh_fmri_val_pred = reg_rh.predict(features_val)

        # Empty correlation array of shape: (LH vertices)
        lh_correlation = np.zeros(lh_fmri_val_pred.shape[1])
        # Correlate each predicted LH vertex with the corresponding ground truth vertex
        for v in range(lh_fmri_val_pred.shape[1]):
            c = corr(lh_fmri_val_pred[:,v], lh_fmri_val[:,v])[0]
            if math.isnan(c) == True:
                lh_correlation[v] = 0
            else:
                lh_correlation[v] = c
        lh_correlation_arrays.append(lh_correlation)

        # Empty correlation array of shape: (RH vertices)
        rh_correlation = np.zeros(rh_fmri_val_pred.shape[1])
        # Correlate each predicted RH vertex with the corresponding ground truth vertex
        for v in range(rh_fmri_val_pred.shape[1]):
            c = corr(rh_fmri_val_pred[:,v], rh_fmri_val[:,v])[0]
            if math.isnan(c) == True:
                rh_correlation[v] = 0
            else:
                rh_correlation[v] = c
        rh_correlation_arrays.append(rh_correlation)

    lh_average_correlation = []    
    rh_average_correlation = []

    for i in range(len(lh_correlation_arrays[0])):
        lh_num = 0
        rh_num = 0
        for j in range (len(lh_correlation_arrays)):
            lh_num += lh_correlation_arrays[j][i];
            rh_num += rh_correlation_arrays[j][i];
        lh_average_correlation.append(lh_num / len(lh_correlation_arrays));
        rh_average_correlation.append(rh_num / len(rh_correlation_arrays));
        
    return lh_average_correlation, rh_average_correlation

cross_val_scores() calls either full or fast cross_validation() and calculates and outputs the median noise-normalized correlation (MNNC) score using the returned arrays of the average correlation scores for each vertex.

In [ ]:
def cross_val_scores(fast, params, folds):
    if(fast == True):
        lh_score, rh_score = fast_cross_validation(params,folds)
    else:
        lh_score, rh_score = cross_validation(params,folds)

    print('\nLH correlation for all verticies: ')
    print(" [ ", lh_score[1], ",", lh_score[2], ",", lh_score[3], ", ... ]")

    result = []
    for i in range (0,len(lh_score)):
        result.append((lh_score[i]*lh_score[i])/0.36)   
    print("LH median noise-normalized score: ")
    print(np.median(result))

    print('\nRH correlation for all verticies: ')
    print(" [ ", rh_score[1], ",", rh_score[2], ",", rh_score[3], ", ... ]")

    result = []
    for i in range (0,len(rh_score)):
        result.append((rh_score[i]*rh_score[i])/0.36)

    print("RH median noise-normalized score: ")
    print(np.median(result))
    
    return lh_score, rh_score

#### Select parameters:

In [ ]:
fast = True # True, False
folds = 3 # 3, 5, 10

#### Cross-validate default linear regression model

In [ ]:
params = ["Ridge",0.05]
lh_score, rh_score = cross_val_scores(fast, params, folds)
params = ["Ridge",0.02]
lh_score, rh_score = cross_val_scores(fast, params, folds)
params = ["Ridge",0.03]
lh_score, rh_score = cross_val_scores(fast, params, folds)

### Visualize averaged correlation score from cross-validation

We map the average correlations for each vertex (lh_score and rh_score), outputted from cross-validating our default Linear Regression model against 10 folds with different train-validation splits, to the brain surface map in fsaverage space. This is the same process as used for mapping and visualizing the fMRI data, with spatial heatmaps used to represent individual vertex's encoding accuracies across the visual cortex.

#### Select parameters:

In [ ]:
hemisphere = 'left' #@param ['left', 'right'] {allow-input: true}

#### Brain Surface Map

In [ ]:
# Load the brain surface map of all vertices
roi_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.all-vertices_fsaverage_space.npy')
fsaverage_all_vertices = np.load(roi_dir)

# Map the correlation results onto the brain surface map
fsaverage_correlation = np.zeros(len(fsaverage_all_vertices))
if hemisphere == 'left':
    fsaverage_correlation[np.where(fsaverage_all_vertices)[0]] = lh_score
elif hemisphere == 'right':
    fsaverage_correlation[np.where(fsaverage_all_vertices)[0]] = rh_score

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_correlation,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title='Encoding accuracy, '+hemisphere+' hemisphere'
    )
view

### Hyperparameter Tuning

By utilising the implemented fast_cross_validation method above, we can test model encoding accuracy for a variety of different models and parameters and compare their median noise-normalized correlation (MNNC) score.

As explained, we first must extract and downsample the image features of all the available training images prior to cross-validating and hyperparameter tuning our models.

In [ ]:
idxs_train = np.arange(len(train_img_list))
train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))
batch_size = 300    
    
all_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_train, transform),
    batch_size=batch_size)

pca = fit_pca(feature_extractor, all_imgs_dataloader)

features = extract_features(feature_extractor, all_imgs_dataloader, pca)

print('\nAll images features:')
print(features.shape)
print('(Training stimulus images × PCA features)')

hyperparameter_tuning() then makes the calls to fast_cross_validation, slightly altering the hyperparameters used for training the model each time. We then take the models with maximum lh and rh MNNC scores and output them along with the parameters used. Linear Regression doesn't have any tuneable hyperparameters, but Lasso and Ridge regression have a regularization, alpha parameter, so we test and tune these linear regression variant models.

We find (as expected) L1 typically performs better closer to 0, while L2 performs better closer to 1. Therefore when selecting parameters, we start from '0+increment' and increment up for Lasso and from 1 and decrement down for Ridge.

In [ ]:
def hyperparameter_tuning(variant, increment, folds, num_models):
    lh_scores = []
    rh_scores = []

    for i in tqdm(range(num_models)):
        if variant == "Lasso":
            params = [variant, (i+1)*increment]
        elif variant == "Ridge":
            params = [variant, i*increment]
        else:
            print("Error: Invalid 'variant'")
            break
        lh_score, rh_score = fast_cross_validation(params,folds)
        
        result = []
        for i in range (0,len(lh_score)):
            result.append((lh_score[i]*lh_score[i])/0.36)
        lh_scores.append(np.median(result))
        
        result = []
        for i in range (0,len(rh_score)):
            result.append((rh_score[i]*rh_score[i])/0.36)
        rh_scores.append(np.median(result))
    
    if variant == "Lasso":
        best_lh_param = ((lh_scores.index(max(lh_scores)) + 1) * increment)
        best_lh_score = max(lh_scores)
        best_rh_param = ((rh_scores.index(max(rh_scores)) + 1) * increment)
        best_rh_score = max(rh_scores)
    elif variant == "Ridge":
        best_lh_param = 1 - ((lh_scores.index(max(lh_scores))) * increment)
        best_lh_score = max(lh_scores)
        best_rh_param = 1 - ((rh_scores.index(max(rh_scores))) * increment)
        best_rh_score = max(rh_scores)
        
    return best_lh_param, best_lh_score, best_rh_param, best_rh_score, lh_scores, rh_scores

#### Hyperparameter Optimization

We first tested both lasso and ridge regression with a broad range of regularization weights (0.05 - 1) and moderately large step size (0.05), using only a small number of folds. [folds = 3, increment = 0.05, num_models = 20] \
This resulted in:

    Lasso regression:
    Best Parameter (lh): 0.05
    Score:  0.13186117333759984
    Best Parameter (rh): 0.05
    Score:  0.12192197343991548
    
    Ridge regression: 
    Best Parameter (lh):  1.0
    Score:  0.1283576704722752
    Best Parameter (rh):  1.0
    Score:  0.1179841435515414
    
The below cell then shows the focused hyperparameter optimization. \
(NOTE: runtime ~9 hours! Therefore code has been commented out to allow running the entire notebook in a reasonable timeframe.)

(Sample output shown below)

#### Select Parameters: 

In [ ]:
folds = 10
increment = 0.005
num_models = 16

In [ ]:
lh_param, lh_score, rh_param, rh_score, lh_scores, rh_scores = hyperparameter_tuning("Lasso",increment,folds,num_models)
print("Lasso regression: ")
print("Best Parameter (lh): ", lh_param)
print("Score: ", lh_score)
print("Best Parameter (rh): ", rh_param)
print("Score: ", rh_score)

lh_param, lh_score, rh_param, rh_score, lh_scores, rh_scores = hyperparameter_tuning("Ridge",increment,folds,num_models)
print("\nRidge regression: ")
print("Best Parameter (lh): ", lh_param)
print("Score: ", lh_score)
print("Best Parameter (rh): ", rh_param)
print("Score: ", rh_score)

In [ ]:
print(lh_scores)

100%|██████████████████████████████████████████████████████████████████| 10/10 [           ]\
100%|██████████████████████████████████████████████████████████████████| 16/16 [             ]

Lasso regression:

Best Parameter (lh):  
Score:  

Best Parameter (rh):  
Score:  

...

100%|██████████████████████████████████████████████████████████████████| 10/10 [           ] \
100%|██████████████████████████████████████████████████████████████████| 16/16 [             ]

Ridge regression:

Best Parameter (lh):  
Score:  

Best Parameter (rh):  
Score:  

#### Optimally performing model: Lasso

#### Parameters: L1 (alpha) regularizer = 
Estimated MNNC Score = 

### Visualize final model encoding accuracy

Now that we have validated our model and found the optimal hyperparamters, we have an estimate of our best and final model's expected performance for predicting the unseen test data. We can visualize this evaluation by mapping the average correlation scores, obtained from cross-validation, for each vertex in fsaverage space, comparing the model with default parameters against our final model with tuned hyperparameters. We also show each individual ROI encoding accuracy, allowing visual (side-by-side) comparisons.

#### Select Parameters:

In [ ]:
hemisphere = 'left' #'right'

#### Brain Surface Map

In [ ]:
# Load the brain surface map of all vertices
roi_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.all-vertices_fsaverage_space.npy')
fsaverage_all_vertices = np.load(roi_dir)

# Map the correlation results onto the brain surface map
fsaverage_correlation = np.zeros(len(fsaverage_all_vertices))
if hemisphere == 'left':
    fsaverage_correlation[np.where(fsaverage_all_vertices)[0]] = lh_score
elif hemisphere == 'right':
    fsaverage_correlation[np.where(fsaverage_all_vertices)[0]] = rh_score

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_correlation,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title='Encoding accuracy, '+hemisphere+' hemisphere'
    )
view

#### ROI Bar Chart Plots

In [ ]:
# Load the ROI classes mapping dictionaries
roi_mapping_files = ['mapping_prf-visualrois.npy', 'mapping_floc-bodies.npy',
    'mapping_floc-faces.npy', 'mapping_floc-places.npy',
    'mapping_floc-words.npy', 'mapping_streams.npy']
roi_name_maps = []
for r in roi_mapping_files:
    roi_name_maps.append(np.load(os.path.join(args.data_dir, 'roi_masks', r),
        allow_pickle=True).item())

# Load the ROI brain surface maps
lh_challenge_roi_files = ['lh.prf-visualrois_challenge_space.npy',
    'lh.floc-bodies_challenge_space.npy', 'lh.floc-faces_challenge_space.npy',
    'lh.floc-places_challenge_space.npy', 'lh.floc-words_challenge_space.npy',
    'lh.streams_challenge_space.npy']
rh_challenge_roi_files = ['rh.prf-visualrois_challenge_space.npy',
    'rh.floc-bodies_challenge_space.npy', 'rh.floc-faces_challenge_space.npy',
    'rh.floc-places_challenge_space.npy', 'rh.floc-words_challenge_space.npy',
    'rh.streams_challenge_space.npy']
lh_challenge_rois = []
rh_challenge_rois = []
for r in range(len(lh_challenge_roi_files)):
    lh_challenge_rois.append(np.load(os.path.join(args.data_dir, 'roi_masks',
        lh_challenge_roi_files[r])))
    rh_challenge_rois.append(np.load(os.path.join(args.data_dir, 'roi_masks',
        rh_challenge_roi_files[r])))

# Select the correlation results vertices of each ROI
roi_names = []
lh_roi_correlation = []
rh_roi_correlation = []
for r1 in range(len(lh_challenge_rois)):
    for r2 in roi_name_maps[r1].items():
        if r2[0] != 0: # zeros indicate to vertices falling outside the ROI of interest
            roi_names.append(r2[1])
            lh_roi_idx = np.where(lh_challenge_rois[r1] == r2[0])[0]
            rh_roi_idx = np.where(rh_challenge_rois[r1] == r2[0])[0]
            lh_roi_correlation.append(lh_correlation[lh_roi_idx])
            rh_roi_correlation.append(rh_correlation[rh_roi_idx])
roi_names.append('All vertices')
lh_roi_correlation.append(lh_correlation)
rh_roi_correlation.append(rh_correlation)

# Create the plot
lh_median_roi_correlation = [np.median(lh_roi_correlation[r])
    for r in range(len(lh_roi_correlation))]
rh_median_roi_correlation = [np.median(rh_roi_correlation[r])
    for r in range(len(rh_roi_correlation))]
plt.figure(figsize=(18,6))
x = np.arange(len(roi_names))
width = 0.30
plt.bar(x - width/2, lh_median_roi_correlation, width, label='Left Hemisphere')
plt.bar(x + width/2, rh_median_roi_correlation, width,
    label='Right Hemishpere')
plt.xlim(left=min(x)-.5, right=max(x)+.5)
plt.ylim(bottom=0, top=1)
plt.xlabel('ROIs')
plt.xticks(ticks=x, labels=roi_names, rotation=60)
plt.ylabel('Median Pearson\'s $r$')
plt.legend(frameon=True, loc=1);

## Challenge submission

Finally, we train and fit the final model on all the available training data, originally supplied by the NSD dataset. This is the our best possible, validated, linear regression model, with optimised parameters and maximally trained feature weights for predicting fMRI repsonses to unseen visual stimuli. 

We then use our final model to output fMRI predictions for all verticies across all brain ROIs for all 8 subjects, before saving and submitting them through the offical Algonauts submission link. We are then given a ranking and challenge (MNNC) score for our model output.

In [ ]:
idxs_train = np.arange(len(train_img_list))
idxs_test = np.arange(len(test_img_list))

train_imgs_dataloader = DataLoader(
        ImageDataset(train_imgs_paths, idxs_train, transform), 
        batch_size=batch_size
    )
test_imgs_dataloader = DataLoader(
    ImageDataset(test_imgs_paths, idxs_test, transform), 
    batch_size=batch_size
)

lh_fmri_train = lh_fmri[idxs_train]
rh_fmri_train = rh_fmri[idxs_train]

del lh_fmri, rh_fmri

pca = fit_pca(feature_extractor, train_imgs_dataloader)

features_train = extract_features(feature_extractor, train_imgs_dataloader, pca)
features_test = extract_features(feature_extractor, test_imgs_dataloader, pca)

del model, pca

reg_lh = LinearRegression().fit(features_train, lh_fmri_train)
reg_rh = LinearRegression().fit(features_train, rh_fmri_train)

lh_fmri_test_pred = reg_lh.predict(features_test)
rh_fmri_test_pred = reg_rh.predict(features_test)

lh_fmri_test_pred = lh_fmri_test_pred.astype(np.float32)
rh_fmri_test_pred = rh_fmri_test_pred.astype(np.float32)

np.save(os.path.join(args.subject_submission_dir, 'lh_pred_test.npy'), lh_fmri_test_pred)
np.save(os.path.join(args.subject_submission_dir, 'rh_pred_test.npy'), rh_fmri_test_pred)

Rank:

Score: 